# Reasoning without Observation
ReWOO proposes an agent that combines a multi-step planner and variable substitution for effective tool use. It was designed to improve on the ReACT-style agent architecture in the following ways:
1. Reduce token consumption and execution time by generating the full chain of tools used in a single pass. _(ReACT style agent architecture requires many LLM calls with redundant prefixes since the system prompt and previous steps are provided to the LLM for each reasoning step)_
2. Simplify the fine-tuning process. Since the planning data doesn't depend on the outputs of the tool, models can be fine-tuned without actually invoking the tools (in theory)

In [2]:
import re
import os
import json
import base64
import asyncio
import platform
import requests
import operator
import playwright
import numpy as np
import pandas as pd
import datetime as dt

from enum import Enum
from typing import List
from typing import Dict
from typing import Tuple
from typing import Union
from typing import Literal
from typing import Optional
from typing import Annotated
from typing import TypedDict
from operator import itemgetter

from IPython import display
from IPython.display import HTML
from IPython.display import Image

from langgraph.graph import END
from langgraph.graph import StateGraph
from langgraph.prebuilt import create_react_agent

from langchain_openai import ChatOpenAI
from langchain_openai import OpenAIEmbeddings

from langchain_core.messages import BaseMessage
from langchain_core.messages.ai import AIMessage
from langchain_core.messages.chat import ChatMessage
from langchain_core.messages.tool import ToolMessage
from langchain_core.messages.human import HumanMessage
from langchain_core.messages.system import SystemMessage
from langchain_core.messages.function import FunctionMessage
from langchain_core.prompts.image import ImagePromptTemplate

from langchain_core.pydantic_v1 import Field
from langchain_core.pydantic_v1 import BaseModel
from langchain_core.runnables import RunnableLambda
from langchain_core.runnables import RunnableParallel
from langchain_core.runnables import RunnablePassthrough
from langchain_core.output_parsers import StrOutputParser
from langchain_core.output_parsers import JsonOutputParser

from langchain_core.runnables.graph import CurveStyle
from langchain_core.runnables.graph import NodeColors
from langchain_core.runnables.graph import MermaidDrawMethod

from langchain import hub
from langchain.schema import Document
from langchain.prompts import PromptTemplate
from langchain.prompts import ChatPromptTemplate
from langchain.prompts import MessagesPlaceholder
from langchain.prompts import HumanMessagePromptTemplate
from langchain.prompts import SystemMessagePromptTemplate
from langchain.agents import create_openai_functions_agent
from langchain.text_splitter import RecursiveCharacterTextSplitter

from langchain_community.vectorstores import Chroma
from langchain_community.document_loaders import WebBaseLoader
from langchain_community.tools.tavily_search import TavilySearchResults

from dotenv import load_dotenv
load_dotenv()
os.environ['LANGCHAIN_TRACING_V2'] = 'true'
os.environ['LANGCHAIN_PROJECT'] = 'ReWOO'

In [3]:
class ReWOO(TypedDict):
    task: str
    plan_string: str
    steps: List
    results: dict
    result: str

In [4]:
model = ChatOpenAI(temperature=0)

In [5]:
prompt = '''For the following task, make plans that can solve the problem step by step. 
For each plan, indicate which external tool together with tool input to retrieve evidence. 
You can store the evidence into a variable #E that can be called by later tools. (Plan, #E1, Plan, #E2, Plan, ...)

Tools can be one of the following:
(1) Google[input]: Worker that searches results from Google. Useful when you need to find short 
and succint answers about a specific topic. The input should be a search query. 
(2) LLM[input]: A pretrained LLM like yourself. Useful when you need to act with general world knowledge and common sense. 
Prioritize it when you are confident in solving the problem yourself. Input can be any instruction.

For example,
Task: Thomas, Toby and Rebecca worked a total of 157 hours in one week. Thomas worked x hours. Toby worked 10 hours less than twice what Thomas worked, 
and Rebecca worked 8 hours less than Toby. How many hours did Rebecca work? 
Plan: Given Thomas worked x hours, translate the problem into algebraic expressions and solve with Wolfram Alpha. #E1 = WolframAlpha[Solve x + (2x - 10) + ((2x - 10) - 8) = 157] 
Plan: Find out the number of hours Thomas worked. #E2 = LLM[What is x, given #E1] 
Plan: Calculate the number of hours Rebecca worked. #E3 = Calculator[(2 * #E2 - 10) - 8] 

Begin!
Describe your plans with rich details. Each plan should be followed by only one #E. 

Task: {task}'''

In [6]:
task = 'what is the hometown of the 2024 australian open winner'
result = model.invoke(prompt.format(task=task))
print(result.content)

Plan: Use Google to search for the hometown of the 2024 Australian Open winner. #E1 = Google[2024 Australian Open winner hometown]


To connect the planner to the graph, we will create a `get_plan` node that accepts the `ReWOO` state and returns with a state update for the `steps` and `plan_string` fields.

In [7]:
regex_pattern = r'Plan:\s*(.+)\s*(#E\d+)\s*=\s*(\w+)\s*\[([^\]]+)\]'
prompt_template = ChatPromptTemplate.from_messages([('user', prompt)])
planner = prompt_template | model

def get_plan(state: ReWOO):
    task = state['task']
    result = planner.invoke({'task': task})
    matches = re.findall(regex_pattern, result.content)
    return {'steps': matches, 'plan_string': result.content}

## Executor
The executor receives the plan and executes the tools in sequence.
Below, instantiate the search engine and define the tool execution node.

In [8]:
search = TavilySearchResults()

In [9]:
def _get_current_task(state: ReWOO):
    if state['results'] is None:
        return 1
    if len(state['results']) == len(state['steps']):
        return None
    else:
        return len(state['results']) + 1
    
def tool_execution(state: ReWOO):
    '''Worker node that executes the tools of a given plan'''
    _step = _get_current_task(state)
    _, step_name, tool, tool_input = state['steps'][_step - 1]
    _results = state['results'] or {}
    for k, v in _results.items():
        tool_input = tool_input.replace(k, v)
    if tool == 'Google':
        result = search.invoke(tool_input)
    elif tool == 'LLM':
        result = model.invoke(tool_input)
    else:
        raise ValueError
    _results[step_name] = str(result)
    return {'results': _results}

## Solver
The solver receives the full plan and generates the final response based on the responses of the tool calls from the worker.

In [10]:
solve_prompt = '''Solve the following task or problem. To solve the problem, we have made step-by-step Plan and retrieved corresponding Evidence to each Plan. 
Use them with caution since long evidence might contain irrelevant information.

{plan}

Now solve the question or task according to provided Evidence above. Respond with the answer directly with no extra words.

Task: {task}
Response:'''

def solve(state: ReWOO):
    plan = ''
    for _plan, step_name, tool, tool_input in state['steps']:
        _results = state['results'] or {}
        for k, v in _results.items():
            tool_input = tool_input.replace(k, v)
            step_name = step_name.replace(k, v)

        plan += solve_prompt.format(plan=plan, task=state['task'])
        result = model.invoke(prompt)
        return {'result': result.content}

## Define Graph
Our graph defines the workflow. Each of the planner, tool executor and solver modules are added as nodes.

In [11]:
def _route(state):
    _step = _get_current_task(state)
    if _step is None:
        return 'solve'
    else:
        return 'tool'

In [13]:
graph = StateGraph(ReWOO)
graph.add_node('plan', get_plan)
graph.add_node('tool', tool_execution)
graph.add_node('solve', solve)
graph.add_edge('plan', 'tool')
graph.add_edge('solve', END)
graph.add_conditional_edges('tool', _route)
graph.set_entry_point('plan')

app = graph.compile()

In [14]:
for s in app.stream({'task': task}):
    print(s)
    print('----')

{'plan': {'plan_string': 'Plan: Use Google to search for the hometown of the 2024 Australian Open winner. #E1 = Google[2024 Australian Open winner hometown] \nPlan: Identify the name of the 2024 Australian Open winner. #E2 = LLM[Who won the 2024 Australian Open, given #E1] \nPlan: Find the hometown of the 2024 Australian Open winner. #E3 = Google[#E2 hometown]', 'steps': [('Use Google to search for the hometown of the 2024 Australian Open winner. ', '#E1', 'Google', '2024 Australian Open winner hometown'), ('Identify the name of the 2024 Australian Open winner. ', '#E2', 'LLM', 'Who won the 2024 Australian Open, given #E1'), ('Find the hometown of the 2024 Australian Open winner. ', '#E3', 'Google', '#E2 hometown')]}}
----
{'tool': {'results': {'#E1': '[{\'url\': "https://en.wikipedia.org/wiki/2024_Australian_Open_–_Men\'s_singles_final", \'content\': "This was the first Australian Open final since 2005 not to feature any of the Big Three members.[5]\\nMedvedev set an Open Era record f

In [16]:
print(s['solve']['result'])

Plan: Use the Pythagorean theorem to find the length of the hypotenuse of a right triangle with legs of length 3 and 4. #E1 = LLM[Apply the Pythagorean theorem with legs of length 3 and 4]

Plan: Calculate the square of each leg, add them together, and take the square root to find the length of the hypotenuse. #E2 = Calculator[Sqrt(3^2 + 4^2)]
